##### Copyright 2019 Die TensorFlow-Autoren.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Zeitreihenvorhersage

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/time_series"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Auf TensorFlow.org anzeigen</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/time_series.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">In Google Colab ausführen</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/time_series.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/time_series.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notizbuch herunterladen</a></td>
</table>

Dieses Tutorial ist eine Einführung in die Zeitreihenprognose mit TensorFlow. Es erstellt einige verschiedene Arten von Modellen, einschließlich Convolutional and Recurrent Neural Networks (CNNs und RNNs).

Dies wird in zwei Hauptteilen mit Unterabschnitten behandelt:

- Prognose für einen einzelnen Zeitschritt:
    - Ein einziges Merkmal.
    - Alle Features.
- Prognostizieren Sie mehrere Schritte:
    - Single-Shot: Treffen Sie alle Vorhersagen auf einmal.
    - Autoregressiv: Machen Sie jeweils eine Vorhersage und leiten Sie die Ausgabe zurück an das Modell.

## Konfiguration

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Der Wetterdatensatz

Dieses Tutorial verwendet einen <a href="https://www.bgc-jena.mpg.de/wetter/" class="external">Wetterzeitreihen-Datensatz</a> , der vom <a href="https://www.bgc-jena.mpg.de" class="external">Max-Planck-Institut für Biogeochemie</a> aufgezeichnet wurde.

Dieser Datensatz enthält 14 verschiedene Merkmale wie Lufttemperatur, Luftdruck und Luftfeuchtigkeit. Diese wurden ab 2003 alle 10 Minuten erfasst. Aus Effizienzgründen verwenden Sie nur die zwischen 2009 und 2016 erfassten Daten. Dieser Abschnitt des Datensatzes wurde von François Chollet für sein Buch <a href="https://www.manning.com/books/deep-learning-with-python" class="external">Deep Learning with Python</a> erstellt.

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

Dieses Tutorial befasst sich nur mit **stündlichen Vorhersagen** . Beginnen Sie also damit, die Daten von 10-Minuten-Intervallen auf 1-Stunden-Intervalle zu subsampeln:

In [ ]:
df = pd.read_csv(csv_path)
# Slice [start:stop:step], starting from index 5 take every 6th record.
df = df[5::6]

date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

Werfen wir einen Blick auf die Daten. Hier sind die ersten Zeilen:

In [ ]:
df.head()

Hier ist die Entwicklung einiger Funktionen im Laufe der Zeit:

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

### Inspizieren und reinigen

Sehen Sie sich als Nächstes die Statistiken des Datensatzes an:

In [ ]:
df.describe().transpose()

#### Windgeschwindigkeit

Eine Sache, die auffallen sollte, ist der `min` Wert der Windgeschwindigkeitsspalte ( `wv (m/s)` ) und der maximale Wert ( `max. wv (m/s)` ). Diese `-9999` ist wahrscheinlich falsch.

Es gibt eine separate Windrichtungsspalte, daher sollte die Geschwindigkeit größer als Null sein ( `>=0` ). Ersetzen Sie es durch Nullen:

In [ ]:
wv = df['wv (m/s)']
bad_wv = wv == -9999.0
wv[bad_wv] = 0.0

max_wv = df['max. wv (m/s)']
bad_max_wv = max_wv == -9999.0
max_wv[bad_max_wv] = 0.0

# The above inplace edits are reflected in the DataFrame.
df['wv (m/s)'].min()

### Feature-Engineering

Bevor Sie mit dem Erstellen eines Modells beginnen, ist es wichtig, Ihre Daten zu verstehen und sicherzustellen, dass Sie die Modelldaten in angemessen formatierten Daten übergeben.

#### Wind

Die letzte Spalte der Daten, `wd (deg)` – gibt die Windrichtung in Gradeinheiten an. Winkel sind keine guten Modelleingaben: 360° und 0° sollten nahe beieinander liegen und reibungslos umlaufen. Die Richtung sollte keine Rolle spielen, wenn der Wind nicht weht.

Im Moment sieht die Verteilung der Winddaten so aus:

In [ ]:
plt.hist2d(df['wd (deg)'], df['wv (m/s)'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]')

Dies ist jedoch für das Modell einfacher zu interpretieren, wenn Sie die Windrichtungs- und -geschwindigkeitsspalten in einen **Windvektor** umwandeln:

In [ ]:
wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')

# Convert to radians.
wd_rad = df.pop('wd (deg)')*np.pi / 180

# Calculate the wind x and y components.
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

# Calculate the max wind x and y components.
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

Die Verteilung der Windvektoren ist für das Modell viel einfacher richtig zu interpretieren:

In [ ]:
plt.hist2d(df['Wx'], df['Wy'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind X [m/s]')
plt.ylabel('Wind Y [m/s]')
ax = plt.gca()
ax.axis('tight')

#### Zeit

In ähnlicher Weise ist die Spalte `Date Time` sehr nützlich, jedoch nicht in dieser Zeichenfolgenform. Beginnen Sie damit, es in Sekunden umzuwandeln:

In [ ]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)

Ähnlich wie die Windrichtung ist die Zeit in Sekunden keine nützliche Modelleingabe. Da es sich um Wetterdaten handelt, haben sie eine klare tägliche und jährliche Periodizität. Es gibt viele Möglichkeiten, wie Sie mit der Periodizität umgehen können.

Sie können brauchbare Signale erhalten, indem Sie Sinus- und Kosinustransformationen verwenden, um die Signale "Tageszeit" und "Jahreszeit" zu löschen:

In [ ]:
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
plt.plot(np.array(df['Day sin'])[:25])
plt.plot(np.array(df['Day cos'])[:25])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

Dadurch erhält das Modell Zugriff auf die wichtigsten Frequenzmerkmale. In diesem Fall wussten Sie vorher, welche Frequenzen wichtig sind.

Wenn Sie diese Informationen nicht haben, können Sie bestimmen, welche Frequenzen wichtig sind, indem Sie Merkmale mit <a href="https://en.wikipedia.org/wiki/Fast_Fourier_transform" class="external">Fast Fourier Transform</a> extrahieren. Um die Annahmen zu überprüfen, hier das `tf.signal.rfft` der Temperatur über der Zeit. Beachten Sie die offensichtlichen Spitzen bei Frequenzen nahe `1/year` und `1/day` :


In [ ]:
fft = tf.signal.rfft(df['T (degC)'])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(df['T (degC)'])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

### Teilen Sie die Daten auf

Sie verwenden eine Aufteilung `(70%, 20%, 10%)` für die Trainings-, Validierungs- und Testdatensätze. Beachten Sie, dass die Daten vor dem Teilen **nicht** zufällig gemischt werden. Dies aus zwei Gründen:

1. Es stellt sicher, dass das Zerhacken der Daten in Fenster aufeinanderfolgender Abtastungen weiterhin möglich ist.
2. Es stellt sicher, dass die Validierungs-/Testergebnisse realistischer sind, da sie anhand der Daten ausgewertet werden, die nach dem Trainieren des Modells gesammelt wurden.

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

### Normalisieren Sie die Daten

Es ist wichtig, Merkmale zu skalieren, bevor ein neuronales Netzwerk trainiert wird. Normalisierung ist eine gängige Methode, um diese Skalierung durchzuführen: Subtrahieren Sie den Mittelwert und dividieren Sie ihn durch die Standardabweichung jedes Merkmals.

Der Mittelwert und die Standardabweichung sollten nur anhand der Trainingsdaten berechnet werden, damit die Modelle keinen Zugriff auf die Werte in den Validierungs- und Testsätzen haben.

Es kann auch argumentiert werden, dass das Modell beim Training keinen Zugriff auf zukünftige Werte im Trainingssatz haben sollte und dass diese Normalisierung mithilfe von gleitenden Durchschnitten erfolgen sollte. Das ist nicht der Schwerpunkt dieses Tutorials, und die Validierungs- und Testsets stellen sicher, dass Sie (etwas) ehrliche Metriken erhalten. Aus Gründen der Einfachheit verwendet dieses Tutorial daher einen einfachen Durchschnitt.

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

Schauen Sie sich nun die Verteilung der Funktionen an. Einige Merkmale haben zwar lange Ausläufer, aber es gibt keine offensichtlichen Fehler wie den Windgeschwindigkeitswert von `-9999` .

In [ ]:
df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

## Datenfensterung

Die Modelle in diesem Lernprogramm erstellen eine Reihe von Vorhersagen basierend auf einem Fenster aufeinanderfolgender Stichproben aus den Daten.

Die Hauptmerkmale der Eingabefenster sind:

- Die Breite (Anzahl der Zeitschritte) der Eingabe- und Beschriftungsfenster.
- Der Zeitversatz zwischen ihnen.
- Welche Features werden als Eingaben, Labels oder beides verwendet?

Dieses Tutorial erstellt eine Vielzahl von Modellen (einschließlich linearer, DNN-, CNN- und RNN-Modelle) und verwendet sie für beide:

- *Single-Output-* und *Multi-Output-* Vorhersagen.
- Vorhersagen für *einzelne Zeitschritte* und *mehrere Zeitschritte* .

Dieser Abschnitt konzentriert sich auf die Implementierung des Datenfensters, sodass es für alle diese Modelle wiederverwendet werden kann.


Je nach Aufgabenstellung und Modelltyp möchten Sie vielleicht unterschiedliche Datenfenster generieren. Hier sind einige Beispiele:

1. Um beispielsweise eine einzelne Vorhersage 24 Stunden in die Zukunft zu treffen, könnten Sie bei einem 24-Stunden-Verlauf ein Fenster wie das folgende definieren:

![Eine Vorhersage 24 Stunden in die Zukunft.](images/raw_window_24h.png)

1. Ein Modell, das bei sechs Stunden Historie eine Vorhersage eine Stunde in die Zukunft macht, würde ein Fenster wie dieses benötigen:

![Eine Vorhersage eine Stunde in die Zukunft.](images/raw_window_1h.png)

Der Rest dieses Abschnitts definiert eine `WindowGenerator` -Klasse. Diese Klasse kann:

1. Behandeln Sie die Indizes und Offsets wie in den Diagrammen oben gezeigt.
2. Aufteilen von Feature-Fenstern in `(features, labels)` Paare.
3. Plotten Sie den Inhalt der resultierenden Fenster.
4. Generieren Sie mithilfe von `tf.data.Dataset` s effizient Batches dieser Fenster aus den Trainings-, Bewertungs- und Testdaten.

### 1. Indizes und Offsets

Beginnen Sie mit dem Erstellen der `WindowGenerator` -Klasse. Die Methode `__init__` enthält die gesamte notwendige Logik für die Eingabe- und Label-Indizes.

Es verwendet auch die Trainings-, Bewertungs- und Test-DataFrames als Eingabe. Diese werden später in `tf.data.Dataset` von Windows konvertiert.

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

Hier ist der Code zum Erstellen der 2 Fenster, die in den Diagrammen am Anfang dieses Abschnitts gezeigt werden:

In [ ]:
w1 = WindowGenerator(input_width=24, label_width=1, shift=24,
                     label_columns=['T (degC)'])
w1

In [ ]:
w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=['T (degC)'])
w2

### 2. Teilen

Bei einer gegebenen Liste aufeinanderfolgender Eingaben konvertiert die Methode `split_window` diese in ein Fenster mit Eingaben und ein Fenster mit Bezeichnungen.

Das zuvor definierte Beispiel `w2` wird wie folgt aufgeteilt:

![Das anfängliche Fenster enthält alle aufeinanderfolgenden Samples, dies teilt es in Paare (Eingänge, Labels) auf](images/split_window.png)

Dieses Diagramm zeigt nicht die `features` der Daten, aber diese `split_window` Funktion verarbeitet auch die `label_columns` , sodass sie sowohl für die Einzelausgabe- als auch für die Mehrfachausgabebeispiele verwendet werden kann.

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

Versuch es:

In [ ]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

Typischerweise werden Daten in TensorFlow in Arrays gepackt, wobei sich der äußerste Index über Beispiele erstreckt (die „Batch“-Dimension). Die mittleren Indizes sind die Dimension(en) "Zeit" oder "Raum" (Breite, Höhe). Die innersten Indizes sind die Merkmale.

Der obige Code nahm einen Stapel von drei 7-Zeitschrittfenstern mit 19 Merkmalen bei jedem Zeitschritt. Es teilt sie in einen Stapel von 6-mal Schritt 19-Feature-Eingaben und ein 1-mal Schritt 1-Feature-Label auf. Das Label hat nur eine Eigenschaft, da der `WindowGenerator` mit `label_columns=['T (degC)']` initialisiert wurde. Zunächst werden in diesem Lernprogramm Modelle erstellt, die einzelne Ausgabebezeichnungen vorhersagen.

### 3. Grundstück

Hier ist eine Plot-Methode, die eine einfache Visualisierung des geteilten Fensters ermöglicht:

In [ ]:
w2.example = example_inputs, example_labels

In [ ]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

Dieses Diagramm richtet Eingaben, Beschriftungen und (spätere) Vorhersagen basierend auf der Zeit aus, auf die sich das Element bezieht:

In [ ]:
w2.plot()

Sie können die anderen Spalten zeichnen, aber das Beispielfenster `w2` Konfiguration hat nur Beschriftungen für die Spalte `T (degC)` .

In [ ]:
w2.plot(plot_col='p (mbar)')

### 4. Erstellen Sie `tf.data.Dataset` s

Schließlich nimmt diese `make_dataset` Methode einen Zeitreihen-DataFrame und konvertiert ihn mithilfe der `tf.keras.utils.timeseries_dataset_from_array` Funktion in ein `tf.data.Dataset` von `(input_window, label_window)` -Paaren:

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

Das `WindowGenerator` Objekt enthält Trainings-, Validierungs- und Testdaten.

Fügen Sie Eigenschaften für den Zugriff auf sie als `tf.data.Dataset` s hinzu, indem Sie die zuvor definierte Methode `make_dataset` verwenden. Fügen Sie außerdem einen Standard-Beispielstapel für einfachen Zugriff und Plotten hinzu:

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

Jetzt gibt Ihnen das `WindowGenerator` Objekt Zugriff auf die `tf.data.Dataset` Objekte, sodass Sie die Daten einfach durchlaufen können.

Die Eigenschaft `Dataset.element_spec` teilt Ihnen die Struktur, Datentypen und Formen der Datensatzelemente mit.

In [ ]:
# Each element is an (inputs, label) pair.
w2.train.element_spec

Das Iterieren über einen `Dataset` ergibt konkrete Batches:

In [ ]:
for example_inputs, example_labels in w2.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

## Einstufige Modelle

Das einfachste Modell, das Sie auf dieser Art von Daten aufbauen können, ist eines, das den Wert einer einzelnen Funktion vorhersagt – 1 Zeitschritt (eine Stunde) in die Zukunft, basierend nur auf den aktuellen Bedingungen.

Beginnen Sie also damit, Modelle zu erstellen, um den `T (degC)` -Wert eine Stunde in die Zukunft vorherzusagen.

![Sagen Sie den nächsten Zeitschritt voraus](images/narrow_window.png)

Konfigurieren Sie ein `WindowGenerator` -Objekt, um diese Single-Step-Paare `(input, label)` zu erzeugen:

In [ ]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=['T (degC)'])
single_step_window

Das `window` erstellt `tf.data.Dataset` s aus den Trainings-, Validierungs- und Testdatensätzen, sodass Sie problemlos Datenstapel durchlaufen können.


In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

### Grundlinie

Vor dem Bau eines trainierbaren Modells wäre es gut, eine Leistungsbaseline als Vergleichspunkt mit den später komplizierteren Modellen zu haben.

Diese erste Aufgabe besteht darin, die Temperatur eine Stunde in die Zukunft vorherzusagen, wenn der aktuelle Wert aller Merkmale gegeben ist. Die aktuellen Werte beinhalten die aktuelle Temperatur.

Beginnen Sie also mit einem Modell, das nur die aktuelle Temperatur als Vorhersage zurückgibt und „keine Änderung“ vorhersagt. Dies ist eine vernünftige Grundlinie, da sich die Temperatur langsam ändert. Natürlich wird diese Baseline weniger gut funktionieren, wenn Sie eine Vorhersage in der Zukunft treffen.

![Senden Sie die Eingabe an die Ausgabe](images/baseline.png)

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

Instanziieren und bewerten Sie dieses Modell:

In [ ]:
baseline = Baseline(label_index=column_indices['T (degC)'])

baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)

Das hat einige Leistungsmetriken gedruckt, aber diese geben Ihnen kein Gefühl dafür, wie gut das Modell abschneidet.

Der `WindowGenerator` hat eine Plot-Methode, aber die Plots werden mit nur einem einzigen Sample nicht sehr interessant sein.

Erstellen Sie also einen breiteren `WindowGenerator` , der Fenster 24 Stunden lang aufeinanderfolgende Eingaben und Beschriftungen gleichzeitig generiert. Die neue Variable `wide_window` ändert nichts an der Funktionsweise des Modells. Das Modell macht immer noch Vorhersagen eine Stunde in die Zukunft basierend auf einem einzelnen Eingabezeitschritt. Hier verhält sich die `time` wie die `batch` : Jede Vorhersage wird unabhängig gemacht, ohne Wechselwirkung zwischen den Zeitschritten:

In [ ]:
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    label_columns=['T (degC)'])

wide_window

Dieses erweiterte Fenster kann ohne Codeänderungen direkt an dasselbe `baseline` übergeben werden. Dies ist möglich, weil die Eingaben und Beschriftungen die gleiche Anzahl von Zeitschritten haben und die Basislinie die Eingabe nur an die Ausgabe weiterleitet:

![Eine Vorhersage 1 Stunde in die Zukunft, jede Stunde.](images/last_window.png)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)

Beachten Sie beim Zeichnen der Vorhersagen des Basismodells, dass es sich lediglich um die um eine Stunde nach rechts verschobenen Beschriftungen handelt:

In [ ]:
wide_window.plot(baseline)

In den obigen Diagrammen von drei Beispielen wird das Einzelschrittmodell über einen Zeitraum von 24 Stunden ausgeführt. Dies verdient eine Erklärung:

- Die blaue `Inputs` zeigt die Eingangstemperatur bei jedem Zeitschritt. Das Modell erhält alle Funktionen, dieser Plot zeigt nur die Temperatur.
- Die grünen `Labels` zeigen den Zielvorhersagewert. Diese Punkte werden zur Vorhersagezeit angezeigt, nicht zur Eingabezeit. Aus diesem Grund wird der Bereich der Labels relativ zu den Eingängen um 1 Schritt verschoben.
- Die orangefarbenen `Predictions` sind die Vorhersagen des Modells für jeden Ausgabezeitschritt. Wenn das Modell perfekte Vorhersagen machen würde, würden die Vorhersagen direkt auf den `Labels` landen.

### Lineares Modell

Das einfachste **trainierbare** Modell, das Sie auf diese Aufgabe anwenden können, besteht darin, eine lineare Transformation zwischen Eingabe und Ausgabe einzufügen. In diesem Fall hängt die Ausgabe eines Zeitschritts nur von diesem Schritt ab:

![Eine Einzelschrittvorhersage](images/narrow_window.png)

Ein `tf.keras.layers.Dense` Layer ohne `activation` ist ein lineares Modell. Die Ebene transformiert nur die letzte Achse der Daten von `(batch, time, inputs)` in `(batch, time, units)` ; es wird unabhängig auf jeden Artikel über die `batch` und `time` angewendet.

In [ ]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', single_step_window.example[0].shape)
print('Output shape:', linear(single_step_window.example[0]).shape)

Dieses Tutorial trainiert viele Modelle, packen Sie also das Trainingsverfahren in eine Funktion:

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

Trainieren Sie das Modell und bewerten Sie seine Leistung:

In [ ]:
history = compile_and_fit(linear, single_step_window)

val_performance['Linear'] = linear.evaluate(single_step_window.val)
performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)

Wie das `baseline` kann das lineare Modell für Stapel von breiten Fenstern aufgerufen werden. Auf diese Weise verwendet, erstellt das Modell eine Reihe unabhängiger Vorhersagen für aufeinanderfolgende Zeitschritte. Die `time` verhält sich wie eine weitere `batch` . Es gibt keine Wechselwirkungen zwischen den Vorhersagen bei jedem Zeitschritt.

![Eine Einzelschrittvorhersage](images/wide_window.png)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)

Hier ist die Darstellung der Beispielvorhersagen für `wide_window` . Beachten Sie, dass die Vorhersage in vielen Fällen eindeutig besser ist als nur die Eingabetemperatur zurückzugeben, aber in einigen Fällen schlechter ist:

In [ ]:
wide_window.plot(linear)

Ein Vorteil linearer Modelle besteht darin, dass sie relativ einfach zu interpretieren sind. Sie können die Gewichte der Ebene herausziehen und das jeder Eingabe zugewiesene Gewicht visualisieren:

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=linear.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

Manchmal legt das Modell nicht einmal das größte Gewicht auf die Eingabe `T (degC)` . Dies ist eines der Risiken der zufälligen Initialisierung. 

### Dicht

Bevor Sie Modelle anwenden, die tatsächlich mit mehreren Zeitschritten arbeiten, sollten Sie die Leistung von tieferen, leistungsfähigeren Modellen mit einzelnen Eingabeschritten überprüfen.

Hier ist ein Modell, das dem `linear` Modell ähnlich ist, außer dass es mehrere `Dense` Schichten zwischen der Eingabe und der Ausgabe stapelt: 

In [ ]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

history = compile_and_fit(dense, single_step_window)

val_performance['Dense'] = dense.evaluate(single_step_window.val)
performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)

### Mehrstufig dicht

Ein Einzelzeitschrittmodell hat keinen Kontext für die aktuellen Werte seiner Eingaben. Es kann nicht sehen, wie sich die Eingabefunktionen im Laufe der Zeit ändern. Um dieses Problem zu lösen, benötigt das Modell Zugriff auf mehrere Zeitschritte, wenn Vorhersagen getroffen werden:

![Für jede Vorhersage werden drei Zeitschritte verwendet.](images/conv_window.png)


Die `baseline` , `linear` und `dense` Modelle behandelten jeden Zeitschritt unabhängig. Hier benötigt das Modell mehrere Zeitschritte als Eingabe, um eine einzelne Ausgabe zu erzeugen.

Erstellen Sie einen `WindowGenerator` , der Stapel von dreistündigen Eingaben und einstündigen Labels erzeugt:

Beachten Sie, dass der `shift` von `Window` relativ zum Ende der beiden Fenster ist.


In [ ]:
CONV_WIDTH = 3
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=['T (degC)'])

conv_window

In [ ]:
conv_window.plot()
plt.title("Given 3 hours of inputs, predict 1 hour into the future.")

Sie könnten ein `dense` Modell in einem Fenster mit mehreren Eingabeschritten trainieren, indem `tf.keras.layers.Flatten` als erste Ebene des Modells hinzufügen:

In [ ]:
multi_step_dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

In [ ]:
print('Input shape:', conv_window.example[0].shape)
print('Output shape:', multi_step_dense(conv_window.example[0]).shape)

In [ ]:
history = compile_and_fit(multi_step_dense, conv_window)

IPython.display.clear_output()
val_performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.val)
performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.test, verbose=0)

In [ ]:
conv_window.plot(multi_step_dense)

Der Hauptnachteil dieses Ansatzes besteht darin, dass das resultierende Modell nur auf Eingabefenstern mit genau dieser Form ausgeführt werden kann. 

In [ ]:
print('Input shape:', wide_window.example[0].shape)
try:
  print('Output shape:', multi_step_dense(wide_window.example[0]).shape)
except Exception as e:
  print(f'\n{type(e).__name__}:{e}')

Die Faltungsmodelle im nächsten Abschnitt beheben dieses Problem.

### Neuronales Faltungsnetzwerk

Eine Faltungsschicht ( `tf.keras.layers.Conv1D` ) verwendet ebenfalls mehrere Zeitschritte als Eingabe für jede Vorhersage.

Unten ist **dasselbe** Modell wie `multi_step_dense` , neu geschrieben mit einer Faltung.

Beachten Sie die Änderungen:

- Die `tf.keras.layers.Flatten` und die erste `tf.keras.layers.Dense` werden durch eine `tf.keras.layers.Conv1D` ersetzt.
- Das `tf.keras.layers.Reshape` ist nicht mehr notwendig, da die Faltung die Zeitachse in ihrer Ausgabe behält.

In [ ]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

Führen Sie es in einem Beispielstapel aus, um zu überprüfen, ob das Modell Ausgaben mit der erwarteten Form erzeugt:

In [ ]:
print("Conv model on `conv_window`")
print('Input shape:', conv_window.example[0].shape)
print('Output shape:', conv_model(conv_window.example[0]).shape)

Trainieren und bewerten Sie es im `conv_window` und es sollte eine ähnliche Leistung wie das `multi_step_dense` Modell erbringen.

In [ ]:
history = compile_and_fit(conv_model, conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(conv_window.val)
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

Der Unterschied zwischen diesem `conv_model` und dem `multi_step_dense` Modell besteht darin, dass das `conv_model` mit Eingaben beliebiger Länge ausgeführt werden kann. Die Faltungsschicht wird auf ein gleitendes Eingabefenster angewendet:

![Ausführen eines Faltungsmodells auf einer Sequenz](images/wide_conv_window.png)

Wenn Sie es mit einer breiteren Eingabe ausführen, wird eine breitere Ausgabe erzeugt:

In [ ]:
print("Wide window")
print('Input shape:', wide_window.example[0].shape)
print('Labels shape:', wide_window.example[1].shape)
print('Output shape:', conv_model(wide_window.example[0]).shape)

Beachten Sie, dass die Ausgabe kürzer als die Eingabe ist. Damit das Training oder Plotten funktioniert, müssen die Beschriftungen und die Vorhersage dieselbe Länge haben. Erstellen Sie also einen `WindowGenerator` , um breite Fenster mit ein paar zusätzlichen Eingabezeitschritten zu erzeugen, damit die Label- und Vorhersagelängen übereinstimmen: 

In [ ]:
LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=1,
    label_columns=['T (degC)'])

wide_conv_window

In [ ]:
print("Wide conv window")
print('Input shape:', wide_conv_window.example[0].shape)
print('Labels shape:', wide_conv_window.example[1].shape)
print('Output shape:', conv_model(wide_conv_window.example[0]).shape)

Jetzt können Sie die Vorhersagen des Modells in einem breiteren Fenster darstellen. Beachten Sie die 3 Eingabezeitschritte vor der ersten Vorhersage. Jede Vorhersage hier basiert auf den 3 vorangegangenen Zeitschritten:

In [ ]:
wide_conv_window.plot(conv_model)

### Wiederkehrendes neuronales Netzwerk

Ein Recurrent Neural Network (RNN) ist eine Art von neuronalem Netzwerk, das sich gut für Zeitreihendaten eignet. RNNs verarbeiten eine Zeitreihe Schritt für Schritt und behalten einen internen Zustand von Zeitschritt zu Zeitschritt bei.

Weitere Informationen finden Sie im Tutorial [Textgenerierung mit einem RNN](https://www.tensorflow.org/text/tutorials/text_generation) und im Leitfaden [Recurrent Neural Networks (RNN) with Keras](https://www.tensorflow.org/guide/keras/rnn) .

In diesem Lernprogramm verwenden Sie eine RNN-Schicht namens Long Short-Term Memory ( `tf.keras.layers.LSTM` ).

Ein wichtiges Konstruktorargument für alle Keras-RNN-Layer, wie z. B. `tf.keras.layers.LSTM` , ist das Argument `return_sequences` . Diese Einstellung kann den Layer auf zwei Arten konfigurieren:

1. Bei `False` , dem Standardwert, gibt die Ebene nur die Ausgabe des letzten Zeitschritts zurück, sodass das Modell Zeit hat, seinen internen Zustand aufzuwärmen, bevor es eine einzelne Vorhersage trifft:

![Ein LSTM, das sich aufwärmt und eine einzige Vorhersage macht](images/lstm_1_window.png)

1. Wenn `True` , gibt der Layer eine Ausgabe für jede Eingabe zurück. Dies ist nützlich für:
    - Stapeln von RNN-Schichten.
    - Trainieren eines Modells in mehreren Zeitschritten gleichzeitig.

![Ein LSTM, das nach jedem Zeitschritt eine Vorhersage macht](images/lstm_many_window.png)

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

Mit `return_sequences=True` kann das Modell mit 24 Stunden Daten gleichzeitig trainiert werden.

Hinweis: Dies gibt einen pessimistischen Überblick über die Leistung des Modells. Beim ersten Zeitschritt hat das Modell keinen Zugriff auf vorherige Schritte und kann daher nicht besser abschneiden als die zuvor gezeigten einfachen `linear` und `dense` Modelle.

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', lstm_model(wide_window.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
wide_window.plot(lstm_model)

### Leistung

Mit diesem Datensatz schneidet typischerweise jedes der Modelle etwas besser ab als das davor:

In [ ]:
x = np.arange(len(performance))
width = 0.3
metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]

plt.ylabel('mean_absolute_error [T (degC), normalized]')
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
_ = plt.legend()

In [ ]:
for name, value in performance.items():
  print(f'{name:12s}: {value[1]:0.4f}')

### Modelle mit mehreren Ausgängen

Die Modelle haben bisher alle ein einzelnes Ausgabemerkmal `T (degC)` für einen einzelnen Zeitschritt vorhergesagt.

Alle diese Modelle können konvertiert werden, um mehrere Features vorherzusagen, indem Sie einfach die Anzahl der Einheiten in der Ausgabeschicht ändern und die Trainingsfenster so anpassen, dass alle Features in den `labels` enthalten sind ( `example_labels` ):

In [ ]:
single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you 
    # don't set the `label_columns` argument.
    input_width=1, label_width=1, shift=1)

wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1)

for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

Beachten Sie oben, dass die `features` -Achse der Beschriftungen jetzt die gleiche Tiefe wie die Eingaben hat, anstatt `1` .

#### Grundlinie

Dasselbe Basismodell ( `Baseline` ) kann hier verwendet werden, aber dieses Mal werden alle Funktionen wiederholt, anstatt einen bestimmten `label_index` :

In [ ]:
baseline = Baseline()
baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

In [ ]:
val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(wide_window.val)
performance['Baseline'] = baseline.evaluate(wide_window.test, verbose=0)

#### Dicht

In [ ]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=num_features)
])

In [ ]:
history = compile_and_fit(dense, single_step_window)

IPython.display.clear_output()
val_performance['Dense'] = dense.evaluate(single_step_window.val)
performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)

#### RNN


In [ ]:
%%time
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1)

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_features)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val)
performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0)

print()

<a id="residual"></a>

#### Erweitert: Restverbindungen

Das `Baseline` -Modell von früher nutzte die Tatsache, dass sich die Sequenz von Zeitschritt zu Zeitschritt nicht drastisch ändert. Jedes bisher in diesem Tutorial trainierte Modell wurde zufällig initialisiert und musste dann lernen, dass die Ausgabe eine kleine Änderung gegenüber dem vorherigen Zeitschritt ist.

Während Sie dieses Problem durch sorgfältige Initialisierung umgehen können, ist es einfacher, dies in die Modellstruktur zu integrieren.

In der Zeitreihenanalyse ist es üblich, Modelle zu erstellen, die statt den nächsten Wert vorherzusagen, vorhersagen, wie sich der Wert im nächsten Zeitschritt ändern wird. In ähnlicher Weise beziehen sich Restnetzwerke – oder <a href="https://arxiv.org/abs/1512.03385" class="external">ResNets</a> – im Deep Learning auf Architekturen, bei denen jede Schicht zum akkumulierten Ergebnis des Modells beiträgt.

So nutzen Sie das Wissen, dass die Änderung gering sein sollte.

![Ein Modell mit einer Restverbindung](images/residual.png)

Im Wesentlichen initialisiert dies das Modell so, dass es mit der `Baseline` übereinstimmt. Für diese Aufgabe hilft es Modellen, schneller zu konvergieren, mit etwas besserer Leistung.

Dieser Ansatz kann in Verbindung mit jedem in diesem Lernprogramm besprochenen Modell verwendet werden.

Hier wird es auf das LSTM-Modell angewendet, beachten Sie die Verwendung von `tf.initializers.zeros` , um sicherzustellen, dass die anfänglich vorhergesagten Änderungen gering sind und die verbleibende Verbindung nicht überwältigen. Hier gibt es keine symmetriebrechenden Bedenken für die Gradienten, da die `zeros` nur auf der letzten Ebene verwendet werden.

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    return inputs + delta

In [ ]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # The predicted deltas should start small.
        # Therefore, initialize the output layer with zeros.
        kernel_initializer=tf.initializers.zeros())
]))

history = compile_and_fit(residual_lstm, wide_window)

IPython.display.clear_output()
val_performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.val)
performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.test, verbose=0)
print()

#### Leistung

Hier ist die Gesamtleistung für diese Multi-Output-Modelle.

In [ ]:
x = np.arange(len(performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
plt.ylabel('MAE (average over all outputs)')
_ = plt.legend()

In [ ]:
for name, value in performance.items():
  print(f'{name:15s}: {value[1]:0.4f}')

Die oben genannten Leistungen sind über alle Modellausgaben gemittelt.

## Mehrstufige Modelle

Sowohl das Single-Output- als auch das Multiple-Output-Modell in den vorherigen Abschnitten machten **Vorhersagen für einzelne Zeitschritte** , eine Stunde in die Zukunft.

In diesem Abschnitt wird untersucht, wie diese Modelle erweitert werden können, um **mehrere Zeitschrittvorhersagen** zu machen.

Bei einer mehrstufigen Vorhersage muss das Modell lernen, eine Reihe zukünftiger Werte vorherzusagen. Anders als bei einem einstufigen Modell, bei dem nur ein einziger zukünftiger Punkt vorhergesagt wird, sagt ein mehrstufiges Modell eine Folge von zukünftigen Werten voraus.

Dazu gibt es zwei grobe Ansätze:

1. Einzelschussvorhersagen, bei denen die gesamte Zeitreihe auf einmal vorhergesagt wird.
2. Autoregressive Vorhersagen, bei denen das Modell nur Einzelschrittvorhersagen macht und seine Ausgabe als Eingabe zurückgeführt wird.

In diesem Abschnitt sagen alle Modelle **alle Merkmale über alle Ausgabezeitschritte hinweg** voraus.


Für das mehrstufige Modell bestehen die Trainingsdaten wiederum aus stündlichen Stichproben. Hier werden die Modelle jedoch lernen, 24 Stunden in die Zukunft vorherzusagen, wenn sie 24 Stunden in der Vergangenheit liegen.

Hier ist ein `Window` Objekt, das diese Slices aus dem Datensatz generiert:

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_window

### Grundlinien

Eine einfache Grundlage für diese Aufgabe besteht darin, den letzten Eingabezeitschritt für die erforderliche Anzahl von Ausgabezeitschritten zu wiederholen:

![Wiederholen Sie die letzte Eingabe für jeden Ausgabeschritt](images/multistep_last.png)

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(last_baseline)

Da diese Aufgabe darin besteht, 24 Stunden in die Zukunft zu prognostizieren, besteht ein weiterer einfacher Ansatz darin, den Vortag zu wiederholen, vorausgesetzt, morgen wird es ähnlich sein:

![Am Vortag wiederholen](images/multistep_repeat.png)

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.losses.MeanSquaredError(),
                        metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance['Repeat'] = repeat_baseline.evaluate(multi_window.val)
multi_performance['Repeat'] = repeat_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(repeat_baseline)

### Single-Shot-Modelle

Ein High-Level-Ansatz für dieses Problem ist die Verwendung eines "Single-Shot"-Modells, bei dem das Modell die gesamte Sequenzvorhersage in einem einzigen Schritt macht.

Dies kann effizient als `tf.keras.layers.Dense` mit `OUT_STEPS*features` -Ausgabeeinheiten implementiert werden. Das Modell muss diese Ausgabe nur in die erforderliche `(OUTPUT_STEPS, features)` .

#### Linear

Ein einfaches lineares Modell, das auf dem letzten Eingabezeitschritt basiert, schneidet besser ab als jede Basislinie, ist aber zu schwach. Das Modell muss `OUTPUT_STEPS` aus einem einzelnen Eingabezeitschritt mit einer linearen Projektion vorhersagen. Es kann nur einen niedrigdimensionalen Teil des Verhaltens erfassen, der wahrscheinlich hauptsächlich auf der Tages- und Jahreszeit basiert.

![Sagen Sie alle Zeitschritte ab dem letzten Zeitschritt voraus](images/multistep_dense.png)

In [ ]:
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_linear_model)

#### Dicht

Das Hinzufügen eines `tf.keras.layers.Dense` zwischen Eingabe und Ausgabe verleiht dem linearen Modell mehr Leistung, basiert aber immer noch nur auf einem einzigen Eingabezeitschritt.

In [ ]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_dense_model)

#### CNN

Ein Faltungsmodell trifft Vorhersagen auf der Grundlage eines Verlaufs mit fester Breite, was zu einer besseren Leistung als das dichte Modell führen kann, da es sehen kann, wie sich die Dinge im Laufe der Zeit ändern:

![Ein Faltungsmodell sieht, wie sich die Dinge im Laufe der Zeit ändern](images/multistep_conv.png)

In [ ]:
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_conv_model, multi_window)

IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_conv_model)

#### RNN

Ein wiederkehrendes Modell kann lernen, eine lange Historie von Eingaben zu verwenden, wenn dies für die Vorhersagen, die das Modell macht, relevant ist. Hier akkumuliert das Modell den internen Zustand für 24 Stunden, bevor es eine einzelne Vorhersage für die nächsten 24 Stunden macht.

In diesem Single-Shot-Format muss das LSTM nur im letzten Zeitschritt eine Ausgabe erzeugen, also legen `return_sequences=False` in `tf.keras.layers.LSTM` .

![Das LSTM akkumuliert den Zustand über das Eingabefenster und macht eine einzelne Vorhersage für die nächsten 24 Stunden](images/multistep_lstm.png)


In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)

### Erweitert: Autoregressives Modell

Die obigen Modelle sagen alle die gesamte Ausgabesequenz in einem einzigen Schritt voraus.

In manchen Fällen kann es für das Modell hilfreich sein, diese Vorhersage in einzelne Zeitschritte zu zerlegen. Dann kann die Ausgabe jedes Modells bei jedem Schritt in sich selbst zurückgeführt werden, und Vorhersagen können abhängig vom vorherigen gemacht werden, wie im klassischen <a href="https://arxiv.org/abs/1308.0850" class="external">Generieren von Sequenzen mit rekurrenten neuronalen Netzen</a> .

Ein klarer Vorteil dieses Modelltyps besteht darin, dass es so eingerichtet werden kann, dass es eine Ausgabe mit unterschiedlicher Länge erzeugt.

Sie könnten jedes der Einzelschritt-Modelle mit mehreren Ausgaben, die in der ersten Hälfte dieses Tutorials trainiert wurden, nehmen und in einer autoregressiven Rückkopplungsschleife ausführen, aber hier konzentrieren Sie sich auf die Erstellung eines Modells, das explizit dafür trainiert wurde.

![Geben Sie die Ausgabe eines Modells an seine Eingabe zurück](images/multistep_autoregressive.png)

#### RNN

In diesem Lernprogramm wird nur ein autoregressives RNN-Modell erstellt, aber dieses Muster könnte auf jedes Modell angewendet werden, das für die Ausgabe eines einzelnen Zeitschritts ausgelegt ist.

Das Modell hat die gleiche Grundform wie die einstufigen LSTM-Modelle von früher: eine `tf.keras.layers.LSTM` Schicht, gefolgt von einer `tf.keras.layers.Dense` Schicht, die die Ausgaben der `LSTM` -Schicht in Modellvorhersagen umwandelt.

Ein `tf.keras.layers.LSTM` ist eine `tf.keras.layers.LSTMCell` , die in die übergeordnete `tf.keras.layers.RNN` ist, die den Status und die Sequenzergebnisse für Sie verwaltet (Schauen Sie sich die [Recurrent Neural Networks (RNN) mit Keras an](https://www.tensorflow.org/guide/keras/rnn) Anleitung für Details).

In diesem Fall muss das Modell die Eingaben für jeden Schritt manuell verwalten, daher verwendet es `tf.keras.layers.LSTMCell` direkt für die Einzelzeitschrittschnittstelle auf niedrigerer Ebene.

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

Die erste Methode, die dieses Modell benötigt, ist eine `warmup` , um seinen internen Zustand basierend auf den Eingaben zu initialisieren. Einmal trainiert, erfasst dieser Zustand die relevanten Teile der Eingabehistorie. Dies entspricht dem einstufigen `LSTM` -Modell von früher:

In [ ]:
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

Diese Methode gibt eine einzelne Zeitschrittvorhersage und den internen Zustand des `LSTM` zurück:

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

Mit dem Zustand des `RNN` und einer anfänglichen Vorhersage können Sie nun mit der Iteration des Modells fortfahren und die Vorhersagen bei jedem Schritt zurück als Eingabe füttern.

Der einfachste Ansatz zum Sammeln der Ausgabevorhersagen besteht darin, eine Python-Liste und einen `tf.stack` nach der Schleife zu verwenden.

Hinweis: Das Stapeln einer Python-Liste auf diese Weise funktioniert nur mit Eager-Ausführung, mit `Model.compile(..., run_eagerly=True)` für das Training oder mit einer Ausgabe mit fester Länge. Für eine dynamische Ausgabelänge müssten Sie ein `tf.TensorArray` anstelle einer Python-Liste und `tf.range` anstelle des Python `range` verwenden.

In [ ]:
def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the LSTM state.
  prediction, state = self.warmup(inputs)

  # Insert the first prediction.
  predictions.append(prediction)

  # Run the rest of the prediction steps.
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output.
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

Testen Sie dieses Modell mit den Beispieleingaben:

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

Trainieren Sie nun das Modell:

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(feedback_model)

### Leistung

Bei diesem Problem gibt es deutlich abnehmende Renditen als Funktion der Modellkomplexität:

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()

Die Metriken für die Multi-Output-Modelle in der ersten Hälfte dieses Tutorials zeigen die über alle Ausgabefeatures gemittelte Leistung. Diese Leistungen sind ähnlich, aber auch über Ausgabezeitschritte gemittelt. 

In [ ]:
for name, value in multi_performance.items():
  print(f'{name:8s}: {value[1]:0.4f}')

Die erzielten Gewinne beim Übergang von einem dichten Modell zu Faltungs- und rekurrenten Modellen betragen nur wenige Prozent (wenn überhaupt), und das autoregressive Modell schnitt deutlich schlechter ab. Diese komplexeren Ansätze lohnen sich also möglicherweise nicht für **dieses** Problem, aber es gab keine Möglichkeit, dies zu wissen, ohne es zu versuchen, und diese Modelle könnten für **Ihr** Problem hilfreich sein.

## Nächste Schritte

Dieses Tutorial war eine kurze Einführung in die Zeitreihenprognose mit TensorFlow.

Weitere Informationen finden Sie unter:

- Kapitel 15 von <a href="https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/" class="external">Praktisches maschinelles Lernen mit Scikit-Learn, Keras und TensorFlow</a> , 2. Auflage.
- Kapitel 6 von <a href="https://www.manning.com/books/deep-learning-with-python" class="external">Deep Learning mit Python</a> .
- Lektion 8 von <a href="https://www.udacity.com/course/intro-to-tensorflow-for-deep-learning--ud187" class="external">Udacitys Einführung in TensorFlow für Deep Learning</a> , einschließlich der <a href="https://github.com/tensorflow/examples/tree/master/courses/udacity_intro_to_tensorflow_for_deep_learning" class="external">Übungshefte</a> .

Denken Sie auch daran, dass Sie jedes <a href="https://otexts.com/fpp2/index.html" class="external">klassische Zeitreihenmodell</a> in TensorFlow implementieren können – dieses Tutorial konzentriert sich nur auf die integrierten Funktionen von TensorFlow.
